In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

df = pd.read_csv('../datasets/3_data_paragraph_labelled_prepared.csv', encoding='utf-8', encoding_errors='replace')

C:\Users\12406\AppData\Local\Temp\ipykernel_10840\216391889.py:7: DtypeWarning: Columns (1,8,10,11,13,14,15,17,18,27,37,38,63,65,81,114,148,158,166,167) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../datasets/3_data_paragraph_labelled_prepared.csv', encoding='utf-8', encoding_errors='replace')


In [2]:
len(df)

128213

In [3]:
df.head()

,claim,label,question,answer,positive_evidence_text,positive_evidence_title,negative_evidence_text,negative_evidence_title,split,dataset,...,evidence_type,is_auth,evidence_id,evidence_label,article,evidence,entropy,Misinfo_type,speaker,veracity
0,NaN,unreliable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fake_news_corpus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,NaN,fake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fake_news_corpus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,NaN,unreliable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fake_news_corpus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,NaN,unreliable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fake_news_corpus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,NaN,clickbait,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fake_news_corpus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [4]:
# Extract distinct categories with data
categories_with_data = list(df['dataset'].unique())

print(categories_with_data)

['fake_news_corpus', 'fa_kes', 'fakenewsamt', 'banfakenews', 'benjamin_political_news', 'isot_fake_news', 'ti_cnn', 'celebrity', 'ct_fan']


In [8]:
# Print number of entries for each dataset
print(df['dataset'].value_counts())

dataset
banfakenews                58478
isot_fake_news             44898
ti_cnn                     20015
ct_fan                      2462
fa_kes                       804
celebrity                    500
fakenewsamt                  480
benjamin_political_news      326
fake_news_corpus             250
Name: count, dtype: int64


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score

# Dictionary to store f1-scores and classification reports
f1_scores = {}
classification_reports = {}

for dset in categories_with_data:
    # Filter dataset and prepare veracity labels
    temp_df = df[df.dataset == dset]
    temp_df = temp_df[temp_df.veracity != 3]
    temp_df = temp_df[temp_df.veracity.notna()]
    temp_df['veracity'] = temp_df['veracity'].astype(int)
    
    # Remove rows with NaN in article_content
    temp_df = temp_df[temp_df['article_content'].notna()]
    
    # Extract top 40 keywords from text
    vectorizer = CountVectorizer(max_features=40, stop_words='english')
    X = vectorizer.fit_transform(temp_df['article_content'])
    feature_names = vectorizer.get_feature_names_out()
    
    # Convert to DataFrame for easier handling
    keyword_features = pd.DataFrame(X.toarray(), columns=feature_names)
    
    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        keyword_features, 
        temp_df.veracity.values, 
        test_size=0.25, 
        random_state=42
    )
    
    # Train and evaluate model
    clf = RandomForestClassifier(max_depth=20, random_state=0)
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    
    # Calculate and store f1-score
    f1 = f1_score(y_test, preds, average='macro')
    f1_scores[dset] = f1 * 100
    
    # Generate and store classification report
    report = classification_report(y_test, preds)
    classification_reports[dset] = report
    print(f"\nClassification Report for {dset}:")
    print(report)
    print("-" * 80)




Classification Report for fake_news_corpus:
              precision    recall  f1-score   support

           2       1.00      1.00      1.00        51

    accuracy                           1.00        51
   macro avg       1.00      1.00      1.00        51
weighted avg       1.00      1.00      1.00        51

--------------------------------------------------------------------------------

Classification Report for fa_kes:
              precision    recall  f1-score   support

           1       0.59      0.51      0.55       115
           2       0.45      0.52      0.48        86

    accuracy                           0.52       201
   macro avg       0.52      0.52      0.52       201
weighted avg       0.53      0.52      0.52       201

--------------------------------------------------------------------------------

Classification Report for fakenewsamt:
              precision    recall  f1-score   support

           1       0.48      0.61      0.54        57
         

In [7]:
print("\nF1-Scores (Macro Average) by Dataset:")
for dataset, score in f1_scores.items():
    print(f"{dataset}: {score:.1f}%")
    # print("\nClassification Report:")
    # print(classification_reports[dataset])
    # print("-" * 80)


F1-Scores (Macro Average) by Dataset:
fake_news_corpus: 100.0%
fa_kes: 51.5%
fakenewsamt: 45.7%
banfakenews: 91.3%
benjamin_political_news: 72.2%
isot_fake_news: 91.8%
ti_cnn: 89.1%
celebrity: 64.0%
ct_fan: 62.2%
